# Clustering Crypto

In [20]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [21]:
# Use the following endpoint to fetch json data
import json
import pprint
url = "https://min-api.cryptocompare.com/data/all/coinlist"
r = requests.get(url)
content = r.content
content_d = content.decode('utf-8')
data = json.loads(content_d)

In [23]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.


In [ ]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path)
print(crypto_df.shape)
crypto_df.describe()
# Create a DataFrame

### Data Preprocessing

In [ ]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

for col in crypto_df.columns:
    #print(col)
    

crypto_df.rename(columns = {'Unnamed: 0':'Ticker'}, inplace = True)
for col in crypto_df.columns:
    #print(col)
    
crypto_df.set_index("Ticker", inplace=True)
crypto_df.head()

In [ ]:
# Keep only cryptocurrencies that are trading
non_trading = crypto_df[crypto_df["IsTrading"] == False].index
crypto_df.drop(non_trading , inplace=True)

crypto_df['IsTrading'].values.sum()
# crypto_df.head()
crypto_df.shape


In [ ]:
# Keep only cryptocurrencies with a working algorithm
crypto_df= crypto_df[crypto_df['Algorithm']!= 'N/A']

In [ ]:
# Remove the "IsTrading" column
crypto_df.drop(columns=['IsTrading'], inplace=True)
crypto_df.head()

In [ ]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()

In [ ]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df= crypto_df[crypto_df["TotalCoinsMined"]>0]

In [ ]:
# Drop rows where there are 'N/A' text values
crypto_df= crypto_df[crypto_df!='N/A']

In [ ]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
crypto_names=crypto_df['CoinName']

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns='CoinName', inplace=True)

In [ ]:
# Create dummy variables for text features
crypto_dummies= pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_dummies.head()

In [ ]:
# Standardize data
crypto_scaled= StandardScaler().fit_transform(crypto_dummies)
crypto_scaled

### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [ ]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
         data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"]
)
pca_df.head()

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
km = KMeans(n_clusters=4, random_state=0)
# Fit the model
km.fit(pca_df)
# Predict clusters
predictions = km.predict(pca_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pca_df['class'] = km.labels_
pca_df.index = crypto_df.index

In [ ]:
crypto_df.head()
clustered_df = pd.concat([crypto_df, pca_df, crypto_names], axis=1, )
clustered_df.head()

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="class", hover_name="CoinName", hover_data=["Algorithm"])
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=['CoinName', "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"])

In [ ]:
# Print the total number of tradable cryptocurrencies
print(clustered_df.hvplot.table(['CoinName', "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"]))

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype("float")
scaled_two = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
scaled_two = StandardScaler().fit_transform(scaled_two)
scaled_two = pd.DataFrame(scaled_two, columns=['TotalCoinSupply', 'TotalCoinsMined'])

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
scaled_two.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply")